In [254]:
errorTable = "dbo.Error_Table"
auditTable = "dbo.Audit_Log_tbl"
controlTable = "dbo.STG_Control_Table"
audit_data = []
encrypt = "true"
host_name_in_certificate = "*.database.windows.net"

StatementMeta(clintDevpool, 9, 255, Finished, Available)

In [255]:
import msal
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql import functions as F
import datetime as dt

StatementMeta(clintDevpool, 9, 256, Finished, Available)

In [256]:
service_principal_id = "spn-ttia-01-appid"
service_principal_secret = "spn-ttia-01-ServicePrincipalSecret"
tenant_id = "bhf-ttia-tenant-id"
azure_sql_url = "sql-ttia-01-jdbcsqlserver"
database_name = "sql-ttia-01-database"

StatementMeta(clintDevpool, 9, 257, Finished, Available)

In [257]:
def getToken():
    from notebookutils import mssparkutils
    mssparkutils.credentials.getSecret
    
    resource_app_id_url = "https://database.windows.net/.default"
    service_principal_id_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , service_principal_id)
    service_principal_secret_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , service_principal_secret)
    tenant_id_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , tenant_id)
    azure_sql_url_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , azure_sql_url)
    database_name_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , database_name)
    authority = "https://login.microsoftonline.com/" + tenant_id_DB

    # Generating access token using the service principal
    context = msal.ConfidentialClientApplication(service_principal_id_DB, service_principal_secret_DB, authority)
    accounts = context.get_accounts(service_principal_id_DB)
    token = context.acquire_token_silent_with_error([resource_app_id_url], account=None,force_refresh=True)
    if token:
        print("Result Found!")
    if not token:
        print("No suitable token exists in cache. Let's get a new one from AAD.")
        token = context.acquire_token_for_client(resource_app_id_url)

    access_token9 = token["access_token"]

    return access_token9, azure_sql_url_DB, database_name_DB


StatementMeta(clintDevpool, 9, 258, Finished, Available)

In [258]:
def query_table(query):
    tokenvar, azure_sql_url_DB, database_name_DB = getToken()
    df = spark.read \
    .format("jdbc") \
    .option("url", azure_sql_url_DB) \
    .option("databaseName", database_name_DB) \
    .option("accessToken", tokenvar) \
    .option("query",query) \
    .option("encrypt", encrypt) \
    .option("hostNameInCertificate", host_name_in_certificate) \
    .option("msiTokenCacheTtl", 0) \
    .load()
    return df

StatementMeta(clintDevpool, 9, 259, Finished, Available)

In [259]:
def insert_data(df,table):
    tokenvar, azure_sql_url_DB, database_name_DB = getToken()
    df.write.mode("append") \
    .format("jdbc") \
    .option("url", azure_sql_url_DB) \
    .option("dbtable", table) \
    .option("databaseName", database_name_DB) \
    .option("accessToken", tokenvar) \
    .option("encrypt", encrypt) \
    .option("hostNameInCertificate", host_name_in_certificate) \
    .option("msiTokenCacheTtl", 0) \
    .save()
    display(df)

StatementMeta(clintDevpool, 9, 260, Finished, Available)

In [260]:
def createAuditDataframe(audit_data):
    schema = StructType([ \
            StructField("PipelineName",StringType(),True), \
            StructField("RunId",StringType(),True), \
            StructField("CreatedOn",TimestampType(),True), \
            StructField("TableName", StringType(), True), \
            StructField("Filepath",StringType(),True), \
            StructField("Status",StringType(),True), \
            StructField("Errormessage",StringType(),True), \
            StructField("BadRecordsPath", StringType(), True), \
            StructField("StartTime",TimestampType(),True), \
            StructField("EndTime",TimestampType(),True), \
            StructField("CopyDuration",StringType(),True), \
            StructField("DataRead", IntegerType(), True), \
            StructField("DataWritten",IntegerType(),True), \
            StructField("Throughput",StringType(),True), \
            StructField("Fileprocessed",StringType(),True), \
            StructField("ProcessType",StringType(),True), \
          ])    
    
    audit_df = spark.createDataFrame(data=audit_data,schema=schema)
    audit_df.printSchema()
    return audit_df

StatementMeta(clintDevpool, 9, 261, Finished, Available)

In [261]:
def createErrorDataframe(error_data):
    schema = StructType([ \
            StructField("EntityName",StringType(),True), \
            StructField("FileName",StringType(),True), \
            StructField("Status",StringType(),True), \
            StructField("RunId", StringType(), True), \
            StructField("ErrorDescription", StringType(), True), \
            StructField("CreatedOn", TimestampType(), True), \
            StructField("ErrorType", StringType(), True), \
            StructField("BadRecordsPath", StringType(), True), \
          ])    

    error_df = spark.createDataFrame(data=error_data,schema=schema)
    error_df.printSchema()
    return error_df

StatementMeta(clintDevpool, 9, 262, Finished, Available)

In [262]:
# resource_app_id_url = "https://database.windows.net/.default"
# service_principal_id_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , service_principal_id)
# service_principal_secret_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , service_principal_secret)
# tenant_id_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , tenant_id)
# azure_sql_url_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , azure_sql_url)
# database_name_DB = mssparkutils.credentials.getSecretWithLS("LS_AzureKeyVault" , database_name)
# authority = "https://login.microsoftonline.com/" + tenant_id_DB
# context = msal.ConfidentialClientApplication(service_principal_id_DB, service_principal_secret_DB, authority)
# account2 = context.get_accounts(username=None)
# print(account2)

StatementMeta(clintDevpool, 9, 263, Finished, Available)

[]


In [ ]:
# import datetime as dt
# error_data=[]
# createdOn = dt.datetime.now()
# error_data.append(("testentity","","Success","-1191","Staging to Processed Data Load",createdOn,"Staging to Processed Data Load","blah"))
# error_df = createErrorDataframe(error_data)
# insert_data(error_df,errorTable)